In [11]:
# Imports
import pandas as pd
import gradio as gr
from transformers import pipeline

In [40]:
# Data for Named Entity Recognition
with open('Case Dataset/Data files/NER_text_Wikipedia_crawl.txt', 'r') as file:
    named_entity_recognition_data = file.read()

In [14]:
# Data for Translation
translation_data = pd.read_csv('Case Dataset/Data files/Translation_Training.csv', sep=';')

In [23]:
# Data for Summarization
summarization_data = pd.read_csv('Case Dataset/Data files/Summarization_Training.csv', sep=';')

In [3]:
# Models for Named Entitiy Recognition
NAMED_ENTITY_RECOGNITION_MODELS = {
    "BERT (CoNLL-03 English)": "dbmdz/bert-large-cased-finetuned-conll03-english",
    "Wikineural Multilingual (Babelscape)": "Babelscape/wikineural-multilingual-ner",
    "BERT (dslim)": "dslim/bert-large-NER",
    "Biomedical (d4data)": "d4data/biomedical-ner-all"
}

In [4]:
# Models for Summarization
SUMMARIZATION_MODELS = {
    "BART (CNN/DailyMail)": "facebook/bart-large-cnn",
    "T5 (CNN/DailyMail)": "t5-large",
    "Pegasus (Newsroom)": "google/pegasus-newsroom",
    "BART (XSum)": "facebook/bart-large-xsum",
    "T5 (XSum)": "t5-large",
    "Pegasus (XSum)": "google/pegasus-xsum"
}

In [9]:
# Code for Named Entitiy Recognition
def named_entity_recognition(model: str = None, text: str = None):
    model = NAMED_ENTITY_RECOGNITION_MODELS[model]
    ner = pipeline("ner", model=model)
    entities = ner(text)
    return {"text": text, "entities": entities}

In [6]:
# Code for Translation
def helsinki_translation(source_language: str = None, target_language: str = None, text: str = None):
    helsinki_translation_pipeline = pipeline('translation', model=f"Helsinki-NLP/opus-mt-{source_language}-{target_language}")
    helsinki_translation = helsinki_translation_pipeline(text)
    return helsinki_translation[0]['translation_text']

def facebook_translation(source_language: str = None, target_language: str = None, text: str = None):
    facebook_translation_pipeline = pipeline('translation', model=f"facebook/m2m100_418M", src_lang=source_language, tgt_lang=target_language)
    facebook_translation = facebook_translation_pipeline(text)
    return facebook_translation[0]['translation_text']

def translation(source_language: str = None, target_language: str = None, text: str = None):
    first_translation = helsinki_translation(source_language=source_language, target_language=target_language, text=text)
    second_translation = facebook_translation(source_language=source_language, target_language=target_language, text=text)
    return f"{first_translation}\n\n{'─' * 30}\nAlternatives:\n{second_translation}"

In [7]:
# Code for Summarization
def summarization(model: str = None, text: str = None):
    summarization_pipeline = pipeline('summarization', model=SUMMARIZATION_MODELS[model])
    summary = summarization_pipeline(text)
    return summary[0]['summary_text']

In [22]:
# Code for the Gradio Interface
def named_entity_recognition_interface():
    model = gr.Dropdown(list(NAMED_ENTITY_RECOGNITION_MODELS.keys()), label="Model", value="BERT (CoNLL-03 English)")
    text = gr.Textbox(lines=5, label="Input", value="Please enter the text to analyze.")
    interface = gr.Interface(fn=named_entity_recognition, inputs=[model, text], outputs=gr.HighlightedText(), title="Named Entity Recognition")
    return interface

def translation_interface():
    source_language = gr.Dropdown([("English", "en"), ("Spanish", "es"), ("French", "fr"), ("German", "de"), ("Italian", "it")], label="Source Language", multiselect=False, value="en")
    target_language = gr.Dropdown([("English", "en"), ("Spanish", "es"), ("French", "fr"), ("German", "de"), ("Italian", "it")], label="Target Language", multiselect=False, value="de")
    text = gr.Textbox(lines=5, label="Input", value="Please enter the text to translate.")
    interface = gr.Interface(fn=translation, inputs=[source_language, target_language, text], outputs=gr.Textbox() ,title="Translation")
    return interface

def summarization_interface():
    model = gr.Dropdown(list(SUMMARIZATION_MODELS.keys()), label="Model", value="BART (CNN/DailyMail)")
    text = gr.Textbox(lines=5, label="Input", value="Please enter the text to summarize.")
    interface = gr.Interface(fn=summarization, inputs=[model, text], outputs=gr.Textbox(), title="Summarization")
    return interface

def build_interface():
    interface = gr.TabbedInterface([
        named_entity_recognition_interface(),
        translation_interface(),
        summarization_interface()
    ], ["Named Entity Recognition", "Translation", "Summarization"], title="NLP Toolkit")
    return interface

# Main Function
if __name__ == "__main__":
    interface = build_interface()
    interface.launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
